In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
from scipy.stats import mannwhitneyu, ttest_ind
import numpy as np
from scipy.stats import fisher_exact
from sklearn.metrics import roc_auc_score

In [2]:
# Import data
results = pd.read_csv("/path/to/data/synthetic_mrf_neighborhoods_v1.csv")
results

,Unnamed: 0,x,y,CellType,Neighborhood,Image
0,0,72.307489,998.272139,2,A2,B2
1,1,662.245829,449.953921,1,B1,B2
2,2,208.300466,263.468774,1,C2,B2
3,3,775.769326,76.166443,1,B1,B2
4,4,566.821028,137.852305,1,B1,B2
...,...,...,...,...,...,...
1000481,10177,2728.524122,1982.793720,1,B2,A14
1000482,10178,1500.217294,2547.682122,0,A1,A14
1000483,10179,2651.062824,952.325060,1,B2,A14
1000484,10180,2591.233381,2754.023542,2,B2,A14


In [4]:
# Set intermediate path and import neighborhood distance matrix
intermediate_path = "/path/to/intermediates/SyntheticV1/"
dists = pd.read_csv(intermediate_path + "neighborhood_distance_matrix.csv")

In [5]:
# Scale distance matrix
dists = dists.set_index("Unnamed: 0")
dists /= 10000 # Set to bootstrap sample size s
dists

,A0_A1,A0_B2,A0_C1,A10_A1,A10_B2,A10_C1,A11_A1,A11_B2,A11_C1,A12_A1,...,B6_C2,B7_A2,B7_B1,B7_C2,B8_A2,B8_B1,B8_C2,B9_A2,B9_B1,B9_C2
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A0_A1,0.0000,3.9863,2.4016,0.3097,3.6384,2.4498,0.4549,3.5156,2.4690,0.4440,...,2.3227,0.7026,3.3134,2.5522,0.5691,3.4296,2.3105,0.4954,3.1915,2.2405
A0_B2,3.9863,0.0000,4.6649,4.0660,0.4617,4.6817,4.3028,0.5895,4.7399,4.2913,...,4.6598,4.2673,0.9443,4.8539,4.2362,0.9921,4.6692,4.1661,1.1542,4.6038
A0_C1,2.4016,4.6649,0.0000,2.5095,4.2220,0.2798,2.5921,4.1178,0.2086,2.5716,...,0.1921,2.6600,3.9530,0.2948,2.5351,3.9028,0.2139,2.3714,3.6905,0.2777
A10_A1,0.3097,4.0660,2.5095,0.0000,3.7071,2.5429,0.2506,3.5951,2.5639,0.2403,...,2.4392,0.5229,3.4117,2.7013,0.3982,3.5289,2.4566,0.4231,3.2914,2.3650
A10_B2,3.6384,0.4617,4.2220,3.7071,0.0000,4.2274,3.9421,0.2022,4.2918,3.9310,...,4.2417,3.9134,0.7050,4.4334,3.8803,0.8234,4.2513,3.8090,0.8271,4.1777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B8_B1,3.4296,0.9921,3.9028,3.5289,0.8234,4.0940,3.7649,0.7870,4.0562,3.7534,...,3.8749,3.7210,0.2026,4.0512,3.6931,0.0000,3.8877,3.6264,0.2633,3.8881
B8_C2,2.3105,4.6692,0.2139,2.4566,4.2513,0.2729,2.4706,4.1371,0.2161,2.4535,...,0.0684,2.5815,3.8567,0.2607,2.4600,3.8877,0.0000,2.3337,3.6412,0.1212
B9_A2,0.4954,4.1661,2.3714,0.4231,3.8090,2.3152,0.5087,3.7058,2.3514,0.4838,...,2.3389,0.3654,3.5492,2.5666,0.2333,3.6264,2.3337,0.0000,3.3933,2.2533


In [7]:
# Create dataframe with sample and neighborhood information
df = pd.DataFrame({"full":dists.columns})
df["sample"] = [i.split("_")[0] for i in dists.columns]
df["neighborhood"] = [i.split("_")[1] for i in dists.columns]
df["image_neigborhood"] = df["sample"] + "_" + df["neighborhood"]
df

,full,sample,neighborhood,image_neigborhood
0,A0_A1,A0,A1,A0_A1
1,A0_B2,A0,B2,A0_B2
2,A0_C1,A0,C1,A0_C1
3,A10_A1,A10,A1,A10_A1
4,A10_B2,A10,B2,A10_B2
...,...,...,...,...
295,B8_B1,B8,B1,B8_B1
296,B8_C2,B8,C2,B8_C2
297,B9_A2,B9,A2,B9_A2
298,B9_B1,B9,B1,B9_B1


In [8]:
# Dimension reduction with UMAP
import umap
reducer = umap.UMAP(metric = "precomputed", random_state =0)
reduced = reducer.fit_transform(dists.values)
df["UMAP 1"] = reduced[:,0]
df["UMAP 2"] = reduced[:,1]

2024-09-11 10:28:42.159649: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 10:28:42.172270: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 10:28:42.203204: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 10:28:42.203246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 10:28:42.204106: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [9]:
# Color map settings
color_map = {
    'A1': '#1f77b4',  # Dark Blue
    'A2': '#aec7e8',  # Light Blue
    'B1': '#ff7f0e',  # Dark Orange
    'B2': '#ffbb78',  # Light Orange
    'C1': '#2ca02c',  # Dark Green
    'C2': '#98df8a',  # Light Green
}

In [10]:
# Visualize UMAP
fig = px.scatter(df.sort_values("neighborhood"), "UMAP 1", "UMAP 2", color = "neighborhood", color_discrete_map=color_map)
fig.write_html("SyntheticV1_UMAP.html")

In [12]:
# Get long format of distances
dists_long = {"Relationship": [], "Dist": [], "Neighborhood1 Type":[], "Neighborhood2 Type":[]}
for index, row in dists.iterrows():
    for col in dists.columns:
        if index > col:
            if index.split("_")[-1] == col.split("_")[-1]:
                dists_long["Relationship"].append("Same Distribution")
                dists_long["Dist"].append(row[col])
            elif index.split("_")[-1].strip("12") == col.split("_")[-1].strip("12"):
                dists_long["Relationship"].append("Same Marginal")
                dists_long["Dist"].append(row[col])
            else:
                dists_long["Relationship"].append("Different Marginal")
                dists_long["Dist"].append(row[col])
            dists_long["Neighborhood1 Type"].append(index.split("_")[1].strip("12"))
            dists_long["Neighborhood2 Type"].append(col.split("_")[1].strip("12"))
dists_long = pd.DataFrame(dists_long)

sameDist = dists_long[dists_long["Relationship"]=="Same Distribution"]["Dist"]
sameMarg = dists_long[dists_long["Relationship"]=="Same Marginal"]["Dist"]
diffMarg = dists_long[dists_long["Relationship"]=="Different Marginal"]["Dist"]

In [13]:
# Log scale distances
dists_long["Log Dist"] = np.log(dists_long["Dist"])

In [14]:
# Violin plots
fig = px.violin(dists_long, x = "Relationship", y="Log Dist")
fig.update_xaxes(categoryorder='array', categoryarray= ['Same Distribution', 'Same Marginal', 'Different Marginal'])
fig.write_html("SyntheticV1_Violin.html")

In [15]:
# T-test and Mann Whitney U Test
U1, p = mannwhitneyu(sameDist, sameMarg, method="auto", alternative = "less")
print(U1, p)
print(np.mean(sameDist)/np.mean(sameMarg))
print(ttest_ind(sameDist,sameMarg,alternative="less"))

8128431.0 0.0
0.4815988273467972
TtestResult(statistic=-84.0111817041385, pvalue=0.0, df=14848.0)


In [17]:
# AUROC for SMDJ vs. SJ
dists_long = dists_long[dists_long["Relationship"] != "Different Marginal"]
dists_long["Neighborhood Type"] = dists_long["Neighborhood1 Type"]
print(roc_auc_score((dists_long["Relationship"]=="Same Marginal"),dists_long["Dist"]))

0.852545469387755


In [18]:
# Violin plots faceted by neighborhood type
fig = px.violin(dists_long, x = "Relationship", y="Log Dist", facet_col = "Neighborhood Type")
fig.update_traces(width = 0.75)
fig.update_xaxes(categoryorder='array', categoryarray= ['Same Distribution', 'Same Marginal'])
fig.write_html("SyntheticV1_Violin_Faceted.html")

In [19]:
# T-tests and AUROC by facet

for i in ["A","B","C"]:
    sameDist = dists_long[(dists_long["Relationship"]=="Same Distribution")&(dists_long["Neighborhood Type"]==i)]["Dist"]
    sameMarg = dists_long[(dists_long["Relationship"]=="Same Marginal")&(dists_long["Neighborhood Type"]==i)]["Dist"]
    print(ttest_ind(sameDist,sameMarg,alternative="less"))
    filt = dists_long[(dists_long["Neighborhood Type"]==i)]
    print(roc_auc_score((filt["Relationship"]=="Same Marginal"),filt["Dist"]))

TtestResult(statistic=-72.23150859065836, pvalue=0.0, df=4948.0)
0.9250185306122449
TtestResult(statistic=-123.5540301067295, pvalue=0.0, df=4948.0)
0.9828451428571429
TtestResult(statistic=-29.55592435171042, pvalue=3.006360470928134e-177, df=4948.0)
0.7311595918367347


# Compare to Cell Type Enrichment

In [24]:
# Get cell type composition for each image and neighborhood
results = pd.read_csv("/path/to/data/synthetic_mrf_neighborhoods_v1.csv")
results["Count"] = 1
results["image_neighborhood"] = results["Image"] + "_" + results["Neighborhood"]
results = pd.DataFrame(results.groupby(["Image","Neighborhood", "image_neighborhood","CellType"])["Count"].sum()).reset_index()
results = results.pivot(index= ["Image", "Neighborhood", "image_neighborhood"], columns = "CellType", values = "Count")
results = results.fillna(0)
results

CellType                                  0     1     2
Image Neighborhood image_neighborhood                  
A0    A1           A0_A1               2097  1356   876
      B2           A0_B2                805  1516   193
      C1           A0_C1                983   995  1356
A1    A1           A1_A1               2056  1195   795
      B2           A1_B2                960  1700   284
...                                     ...   ...   ...
B8    B1           B8_B1                968  1973   325
      C2           B8_C2                794   805  1099
B9    A2           B9_A2               1998  1250   868
      B1           B9_B1                828  1721   307
      C2           B9_C2                909   908  1211

[300 rows x 3 columns]

In [25]:
# Calculate enrichment p-values and log scale

pvalues = np.zeros_like(results.values)

i = 0
for index, row in results.iterrows():
    result_sample = results[results.index.get_level_values(0) == index[0]]
    j=0
    for col in results.columns:
        tt = row[col]
        tf = np.sum(row.values)-row[col]
        ft = np.sum(result_sample[col])-row[col]
        ff = np.sum(result_sample.values)-np.sum(result_sample[col])-np.sum(row.values) + row[col]
        res = fisher_exact([[tt, tf], [ft, ff]], alternative='greater')
        adj = res.pvalue/(len(results)*len(results.columns))
        if adj != 0:
            pvalues[i][j] = -np.log10(adj)
            print(-np.log10(adj))
        else:
            pvalues[i][j] = 309
        j+=1
    i+=1

77.38210483842592
2.9542425094393248
2.954242509439358
2.9542425094393536
154.28684137396513
2.9542425094393248
2.9542425094393248
2.9542425094393248
165.26349453275813
95.77982206158197
2.9542425094393248
2.9542425094395255
2.954242509439325
150.25672432160565
2.9542425094393248
2.9542425094393248
2.9542425094393248
165.42181486161243
79.2460637016045
2.9542425094393248
2.9542425094395854
2.9542425094393248
154.6552911131477
2.9542425094393248
2.9542425094393248
2.9542425094393248
151.23061408010298
99.80992010236285
2.9542425094393248
2.9542425094393465
2.9542425094393248
158.2834611849612
2.9542425094393248
2.9542425094393248
2.9542425094393248
151.60639767834186
93.07579690035229
2.9542425094393248
2.954242521474471
2.9542425094393248
160.30344391069346
2.9542425094393248
2.9542425094393248
2.9542425094393248
142.02779558115913
88.7412666566963
2.9542425094393248
2.9542425094518374
2.9542425094393248
163.46512375440213
2.9542425094393248
2.9542425094393248
2.9542425094393248
162.68

2.9542425094393248
2.9542425094393248
158.76736625060826
100.69806864200726
2.9542425094393248
2.954242509439325
2.9542425094393248
174.13341316511992
2.9542425094393248
2.9542425094393248
2.9542425094393248
170.00234189450887
73.90879814371722
2.9542425094393248
3.1159580830689233
2.9542425094393248
170.80355041000402
2.9542425094393248
2.9542425094393248
2.9542425094393248
113.17002290435791
94.62164169847046
2.9542425094393248
2.954242509444544
2.9542425094393248
177.70062807812317
2.9542425094393248
2.9542425094393248
2.9542425094393248
152.4220525036671
72.65395315796975
2.9542425094393248
2.9542425094393248
2.9542425094393248
138.81370491506723
2.9542425094393248
2.9542425094393248
2.9542425094393248
132.01265578389837
69.3243102426512
2.9542425094393248
2.9542449900269805
2.9542425094393248
148.3240249391408
2.9542425094393248
2.9542425094393248
2.9542425094393248
131.53349338003233
104.74853103314038
2.9542425094393248
2.9542425094608
2.9542425094393248
187.8709452035741
2.9542

In [26]:
# Collect p-values into distance matrix
dists_enriched = np.zeros((pvalues.shape[0],pvalues.shape[0]))
for i in range(pvalues.shape[0]):
    for j in range(pvalues.shape[0]):
        dists_enriched[i][j] = np.sum(np.abs(pvalues[i]-pvalues[j]))

In [27]:
# Set index and column names to neighborhoods
dists_enriched = pd.DataFrame(dists_enriched, columns = list(results.index.get_level_values(2)), index=list(results.index.get_level_values(2)))

In [28]:
dists_enriched

,A0_A1,A0_B2,A0_C1,A1_A1,A1_B2,A1_C1,A10_A1,A10_B2,A10_C1,A11_A1,...,B6_C2,B7_A2,B7_B1,B7_C2,B8_A2,B8_B1,B8_C2,B9_A2,B9_B1,B9_C2
A0_A1,0.0,227.0,238.0,18.0,223.0,238.0,2.0,227.0,224.0,22.0,...,214.0,35.0,259.0,266.0,26.0,264.0,225.0,8.0,243.0,205.0
A0_B2,227.0,0.0,315.0,245.0,4.0,315.0,229.0,0.0,301.0,249.0,...,291.0,262.0,32.0,343.0,253.0,37.0,302.0,235.0,16.0,282.0
A0_C1,238.0,315.0,0.0,256.0,311.0,0.0,240.0,315.0,14.0,260.0,...,24.0,273.0,347.0,28.0,264.0,352.0,13.0,246.0,331.0,33.0
A1_A1,18.0,245.0,256.0,0.0,241.0,256.0,16.0,245.0,242.0,4.0,...,232.0,17.0,277.0,284.0,8.0,282.0,243.0,10.0,261.0,223.0
A1_B2,223.0,4.0,311.0,241.0,0.0,311.0,225.0,4.0,297.0,245.0,...,287.0,258.0,36.0,339.0,249.0,41.0,298.0,231.0,20.0,278.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B8_B1,264.0,37.0,352.0,282.0,41.0,352.0,266.0,37.0,338.0,286.0,...,328.0,299.0,5.0,380.0,290.0,0.0,339.0,272.0,21.0,319.0
B8_C2,225.0,302.0,13.0,243.0,298.0,13.0,227.0,302.0,1.0,247.0,...,11.0,260.0,334.0,41.0,251.0,339.0,0.0,233.0,318.0,20.0
B9_A2,8.0,235.0,246.0,10.0,231.0,246.0,6.0,235.0,232.0,14.0,...,222.0,27.0,267.0,274.0,18.0,272.0,233.0,0.0,251.0,213.0
B9_B1,243.0,16.0,331.0,261.0,20.0,331.0,245.0,16.0,317.0,265.0,...,307.0,278.0,16.0,359.0,269.0,21.0,318.0,251.0,0.0,298.0


In [29]:
# Long format of distances
dists_enriched_long = {"Relationship": [], "Dist": [], "Neighborhood1 Type":[], "Neighborhood2 Type":[]}
for index, row in dists_enriched.iterrows():
    for col in dists_enriched.columns:
        if index > col:
            if index.split("_")[-1] == col.split("_")[-1]:
                dists_enriched_long["Relationship"].append("Same Distribution")
                dists_enriched_long["Dist"].append(row[col])
            elif index.split("_")[-1].strip("12") == col.split("_")[-1].strip("12"):
                dists_enriched_long["Relationship"].append("Same Marginal")
                dists_enriched_long["Dist"].append(row[col])
            else:
                dists_enriched_long["Relationship"].append("Different Marginal")
                dists_enriched_long["Dist"].append(row[col])
            dists_enriched_long["Neighborhood1 Type"].append(index.split("_")[1].strip("12"))
            dists_enriched_long["Neighborhood2 Type"].append(col.split("_")[1].strip("12"))
dists_enriched_long = pd.DataFrame(dists_enriched_long)

sameDist = dists_enriched_long[dists_enriched_long["Relationship"]=="Same Distribution"]["Dist"]
sameMarg = dists_enriched_long[dists_enriched_long["Relationship"]=="Same Marginal"]["Dist"]
diffMarg = dists_enriched_long[dists_enriched_long["Relationship"]=="Different Marginal"]["Dist"]

In [30]:
# Log scale distances
dists_enriched_long["Log Dist"] = np.log(dists_enriched_long["Dist"]+1)

In [31]:
# Violin plots
fig = px.violin(dists_enriched_long, x = "Relationship", y="Log Dist")
fig.update_xaxes(categoryorder='array', categoryarray= ['Same Distribution', 'Same Marginal', 'Different Marginal'])
fig.write_html("SyntheticV1_Violin_CellEnrichment.html")

In [32]:
# T-test and Mann Whitney U test
U1, p = mannwhitneyu(sameDist, sameMarg, method="auto", alternative = "less")
print(U1, p)
print(np.mean(sameDist)/np.mean(sameMarg))
print(ttest_ind(sameDist,sameMarg,alternative="less"))

25730201.0 1.1292720580652586e-12
0.9159079705897469
TtestResult(statistic=-5.802095852411656, pvalue=3.34127834893723e-09, df=14848.0)


In [33]:
# Table of sample and neighborhoods
df_enriched = pd.DataFrame({"full":dists_enriched.columns})
df_enriched["sample"] = [i.split("_")[0] for i in dists_enriched.columns]
df_enriched["neighborhood"] = [i.split("_")[1] for i in dists_enriched.columns]
df_enriched["image_neigborhood"] = df_enriched["sample"] + "_" + df_enriched["neighborhood"]
df_enriched

,full,sample,neighborhood,image_neigborhood
0,A0_A1,A0,A1,A0_A1
1,A0_B2,A0,B2,A0_B2
2,A0_C1,A0,C1,A0_C1
3,A1_A1,A1,A1,A1_A1
4,A1_B2,A1,B2,A1_B2
...,...,...,...,...
295,B8_B1,B8,B1,B8_B1
296,B8_C2,B8,C2,B8_C2
297,B9_A2,B9,A2,B9_A2
298,B9_B1,B9,B1,B9_B1


In [34]:
# UMAP dimension reduction
reducer = umap.UMAP(metric = "precomputed", random_state=0)
reduced = reducer.fit_transform(dists_enriched.values)
df_enriched["UMAP 1"] = reduced[:,0]
df_enriched["UMAP 2"] = reduced[:,1]

/home/xiongb/.local/lib/python3.10/site-packages/umap/umap_.py:1857: UserWarning:

using precomputed metric; inverse_transform will be unavailable



In [36]:
# Visualize UMAP
fig = px.scatter(df_enriched.sort_values("neighborhood"), "UMAP 1", "UMAP 2", color = "neighborhood", color_discrete_map=color_map)
fig.write_html("SyntheticV1_UMAP_CellEnrichment.html")

In [39]:
# AUROC for SMDJ vs. SJ
dists_enriched_long = dists_enriched_long[dists_enriched_long["Relationship"] != "Different Marginal"]
dists_enriched_long["Neighborhood Type"] = dists_enriched_long["Neighborhood1 Type"]
print(roc_auc_score((dists_enriched_long["Relationship"]=="Same Marginal"),dists_enriched_long["Dist"]))

0.533238984126984


In [40]:
# Violin plots faceted by neighborhood type
fig = px.violin(dists_enriched_long, x = "Relationship", y="Log Dist", facet_col = "Neighborhood Type")
fig.update_traces(width = 0.75)
fig.update_xaxes(categoryorder='array', categoryarray= ['Same Distribution', 'Same Marginal'])
fig.write_html("SyntheticV1_Violin_CellEnrichment_Faceted.html")

In [41]:
# T-test and AUROC faceted by neighborhood type
for i in ["A","B","C"]:
    sameDist = dists_enriched_long[(dists_enriched_long["Relationship"]=="Same Distribution")&(dists_enriched_long["Neighborhood Type"]==i)]["Dist"]
    sameMarg = dists_enriched_long[(dists_enriched_long["Relationship"]=="Same Marginal")&(dists_enriched_long["Neighborhood Type"]==i)]["Dist"]
    print(ttest_ind(sameDist,sameMarg,alternative="less"))
    filt = dists_enriched_long[(dists_enriched_long["Neighborhood Type"]==i)]
    print(roc_auc_score((filt["Relationship"]=="Same Marginal"),filt["Dist"]))

TtestResult(statistic=-0.9623323124564664, pvalue=0.16796486404364563, df=4948.0)
0.515861469387755
TtestResult(statistic=-7.090921573223588, pvalue=7.600199987852518e-13, df=4948.0)
0.5676250612244897
TtestResult(statistic=-1.667960916869042, pvalue=0.04769338070909502, df=4948.0)
0.5157626122448979
